<a href="https://colab.research.google.com/github/wenjunc2/PAHO-Drug-Data-Cleaning-Summer-2022/blob/main/%E6%99%AE%E9%80%9A%E7%BC%96%E7%A8%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1: Set up indexes for clean catagol drug names

In [1]:
# read into clean catagol name
import pandas as pd
CleanDataBase = pd.read_excel('CleanDataBase.xlsx')
CatalogName = CleanDataBase['Catalog Name']
CatalogName = list(set(CatalogName))
CatalogName = [x for x in CatalogName if str(x) != 'nan']

CatalogNameCopy = CatalogName.copy()
# uppercase everything
CatalogNameU = []
for i in CatalogNameCopy:
  CatalogNameU.append(i.upper())

In [2]:
CatalogNameU[:5]

['AMLODIPINO 5 MG + VALSARTAN 80 MG, TABLET, TBD',
 'GLIBENCLAMIDE 5 MG + METFORMIN 500 MG, TABLET, TBD',
 'VILDAGLIPTIN 50 MG + METFORMIN 1000 MG, TABLET, TBD',
 'LOPINAVIR 40 MG + RITONAVIR 10 MG, ORAL PELLETS, 120 CAPSULES BOTTLE',
 'CISPLATIN 50 MG, POWDER FOR INJECTION, 100 ML VIAL, 1X1']

In [3]:
# split the CatalogNameU
#from nltk.tokenize import WordPunctTokenizer  # this function split the string by space
#tokenizer = WordPunctTokenizer()

CatalogNameUS = []
for item in CatalogNameU:
  temp = item.split(",")
  CatalogNameUS.append(temp)

In [4]:
CatalogNameUS[:5] # after split by ","

[['AMLODIPINO 5 MG + VALSARTAN 80 MG', ' TABLET', ' TBD'],
 ['GLIBENCLAMIDE 5 MG + METFORMIN 500 MG', ' TABLET', ' TBD'],
 ['VILDAGLIPTIN 50 MG + METFORMIN 1000 MG', ' TABLET', ' TBD'],
 ['LOPINAVIR 40 MG + RITONAVIR 10 MG',
  ' ORAL PELLETS',
  ' 120 CAPSULES BOTTLE'],
 ['CISPLATIN 50 MG', ' POWDER FOR INJECTION', ' 100 ML VIAL', ' 1X1']]

In [5]:
# summary the list of presentation form
PresentationForm = []

for item in CatalogNameUS:
  if len(item) >= 2:
    PresentationForm.append(item[1])

In [6]:
PresentationForm = list(set(PresentationForm))

len(PresentationForm)

29

In [7]:
PresentationForm[:5] # the presentation form

[' CAPSULE (PRONLONGED RELEASE)',
 ' INHALER',
 ' INJECTION. 500MG/20ML ROAD BOTTLE',
 ' SOLUTION FOR INJECTION',
 ' TABLET (DISPERSIBLE']

In [8]:
# grab the main medicine 
MainMedicine = []

for item in CatalogNameUS:
  MainMedicine.append(item[0])

In [9]:
print(len(MainMedicine), MainMedicine[:5])

462 ['AMLODIPINO 5 MG + VALSARTAN 80 MG', 'GLIBENCLAMIDE 5 MG + METFORMIN 500 MG', 'VILDAGLIPTIN 50 MG + METFORMIN 1000 MG', 'LOPINAVIR 40 MG + RITONAVIR 10 MG', 'CISPLATIN 50 MG']


In [10]:
# split MainMedicine to get only the medicne name, and use the medicine name as the index
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

MainMedicineSpl = []
for item in MainMedicine:
  tokenlist = tokenizer.tokenize(item)
  MainMedicineSpl.append(tokenlist)

In [11]:
MainMedicineSpl[:5]

[['AMLODIPINO', '5', 'MG', '+', 'VALSARTAN', '80', 'MG'],
 ['GLIBENCLAMIDE', '5', 'MG', '+', 'METFORMIN', '500', 'MG'],
 ['VILDAGLIPTIN', '50', 'MG', '+', 'METFORMIN', '1000', 'MG'],
 ['LOPINAVIR', '40', 'MG', '+', 'RITONAVIR', '10', 'MG'],
 ['CISPLATIN', '50', 'MG']]

In [12]:
MainMedicineIndex = []
for item in MainMedicineSpl:
  templist = []
  for subitem in item:
    if len(subitem) > 4 and subitem.isdigit() is False:  ## ignoring these elements such as "MG", "+", ... 
      templist.append(subitem)
  MainMedicineIndex.append(templist)


In [13]:
MainMedicineIndex[:5]

[['AMLODIPINO', 'VALSARTAN'],
 ['GLIBENCLAMIDE', 'METFORMIN'],
 ['VILDAGLIPTIN', 'METFORMIN'],
 ['LOPINAVIR', 'RITONAVIR'],
 ['CISPLATIN']]

In [14]:
CatalogIndexData = {"Catalog Name": CatalogNameU, "Catalog Index": MainMedicineIndex}

In [15]:
CatalogIndexData_df = pd.DataFrame(CatalogIndexData)

In [16]:
len(CatalogIndexData_df)

462

In [17]:
CatalogIndexData_df[:5]

,Catalog Name,Catalog Index
0,"AMLODIPINO 5 MG + VALSARTAN 80 MG, TABLET, TBD","[AMLODIPINO, VALSARTAN]"
1,"GLIBENCLAMIDE 5 MG + METFORMIN 500 MG, TABLET,...","[GLIBENCLAMIDE, METFORMIN]"
2,"VILDAGLIPTIN 50 MG + METFORMIN 1000 MG, TABLET...","[VILDAGLIPTIN, METFORMIN]"
3,"LOPINAVIR 40 MG + RITONAVIR 10 MG, ORAL PELLET...","[LOPINAVIR, RITONAVIR]"
4,"CISPLATIN 50 MG, POWDER FOR INJECTION, 100 ML ...",[CISPLATIN]


## Part 2: get the chile 1 data set drug names list

In [18]:
import chardet
with open('CHLDiab1.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'confidence': 0.73, 'encoding': 'Windows-1252', 'language': ''}

In [19]:
CHLDiab1 = pd.read_csv('CHLDiab1.csv',encoding='Windows-1252')
#CHLDiab1.head(1)

In [20]:
CHLDiab1DirtyName = CHLDiab1['Descripcion linea Adquisicion']

In [21]:
len(CHLDiab1DirtyName) # length of drug names

20606

In [22]:
CHLDiab1DirtyName = list(set(CHLDiab1DirtyName))
len(CHLDiab1DirtyName) # length of drug names after remove duplicates

3663

In [23]:
# uppercase everything
CHLDiab1DirtyNameU = []
for i in CHLDiab1DirtyName:
  CHLDiab1DirtyNameU.append(i.upper())

In [24]:
CHLDiab1DirtyNameU[:5]

['METFORMINA (CLORHIDRATO) COMPRIMIDO RECUBIERTO 850 MG EQUIVALENTE A GLAFORNIL\r\r\n',
 'METFORMINA 1000 MG COMPRIMIDOS DE LIBERACIÓN PROLONGADA. ENVASE 30 COMPRIMIDOS',
 'METFORMINA 1000 COMPRIMIDOS RECUBIERTOS',
 'INSUÑLINA DEGLUDEC 100 UI /ML X 3 ML SIMILAR A TRESIBA FLEX TOUCH',
 '•\tDAPAGLIFLOZINA (COMO PROPANODIOL) 5 MG. Y METFORMINA CLORHIDRATO 1000 MG. FORMA FARMACÉUTICA, COMPRIMIDOS RECUBIERTOS DE LIBERACIÓN PROLONGADA.']

In [25]:
#!pip install googletrans==3.1.0a0
from googletrans import Translator
translator = Translator(service_urls=['translate.googleapis.com'])
example = CHLDiab1DirtyNameU[0]
translation = translator.translate(example, dest='en')
print(translation.text)

METFORMIN (HYDROCHLORIDE) COATED TABLET 850 MG EQUIVALENT TO GLAFORNIL


In [26]:
CHLDiab1DirtyNameU[0]

'METFORMINA (CLORHIDRATO) COMPRIMIDO RECUBIERTO 850 MG EQUIVALENTE A GLAFORNIL\r\r\n'

In [27]:
# get the index about medicine name for each drug name 

#from nltk.tokenize import WordPunctTokenizer  # this function split the string by space
#tokenizer = WordPunctTokenizer()

CHLDiab1DirtyNameUSpl = []
for item in CHLDiab1DirtyNameU:
  tokenlist = tokenizer.tokenize(item)
  CHLDiab1DirtyNameUSpl.append(tokenlist)

In [28]:
CHLDiab1DirtyNameUSpl[:1]

[['METFORMINA',
  '(',
  'CLORHIDRATO',
  ')',
  'COMPRIMIDO',
  'RECUBIERTO',
  '850',
  'MG',
  'EQUIVALENTE',
  'A',
  'GLAFORNIL']]

In [29]:
CHLDiab1DirtyNameIndex = []
for item in CHLDiab1DirtyNameUSpl:
  templist = []
  for subitem in item:
    if len(subitem) > 4 and subitem.isdigit() is False:  ## ignoring these elements such as "MG", "+", ... 
      templist.append(subitem)
  CHLDiab1DirtyNameIndex.append(templist)

In [30]:
CHLDiab1DirtyNameIndex[:1]

[['METFORMINA',
  'CLORHIDRATO',
  'COMPRIMIDO',
  'RECUBIERTO',
  'EQUIVALENTE',
  'GLAFORNIL']]

In [31]:
# translate these names (indexes) to English
CHLDiab1DirtyNameIndexEng = []

for item in CHLDiab1DirtyNameIndex:
  temp_sublist = []
  for subitem in item:
    temp = translator.translate(subitem, dest='en')
    temp_sublist.append(temp.text)
  CHLDiab1DirtyNameIndexEng.append(temp_sublist)

# this block runs 17 mins

## Part 3: compare one durg name with each index and select one index to pair

### part 3 - approach 1 (according to the same vocabularies)

In [69]:
# try to pair the 

myfirstlist = []

for item in CHLDiab1DirtyNameIndexEng[0]:
  for index in MainMedicineIndex:
    add = False
    for subindex in index:
      if subindex == item:
        add = True
    if add == True:
      myfirstlist.append(index)


In [71]:
CHLDiab1DirtyNameIndexEng[0]

['METFORMIN', 'TABLETS', 'RELEASE', 'EXTENDED']

In [70]:
myfirstlist

[['SAXAGLIPTIN', 'METFORMIN'],
 ['SITAGLIPTIN', 'METFORMIN'],
 ['METFORMIN', 'GLIMEPIRIDE'],
 ['GLIBENCLAMIDE', 'METFORMIN'],
 ['LINAGLIPTIN', 'METFORMIN'],
 ['LINAGLIPTIN', 'METFORMIN'],
 ['GLIBENCLAMIDE', 'METFORMIN'],
 ['VILDAGLIPTIN', 'METFORMIN'],
 ['DAPAGLIFLOZIN', 'METFORMIN'],
 ['SITAGLIPTIN', 'METFORMIN'],
 ['EMPAGLIFLOZIN', 'METFORMIN'],
 ['DAPAGLIFLOZIN', 'METFORMIN'],
 ['EMPAGLIFOZIN', 'METFORMIN'],
 ['SITAGLIPTIN', 'METFORMIN'],
 ['GLIMEPIRIDE', 'METFORMIN'],
 ['CANAGLIFOZINE', 'METFORMIN'],
 ['CANAGLIFOZINE', 'METFORMIN'],
 ['EMPAGLIFLOZIN', 'METFORMIN'],
 ['ALOGLIPTIN', 'METFORMIN'],
 ['GLIBENCLAMIDE', 'METFORMIN'],
 ['METFORMIN'],
 ['METFORMIN', 'DAPAGLIFLOZIN'],
 ['METFORMIN'],
 ['ALOGLIPTIN', 'METFORMIN'],
 ['GLIBENCLAMIDE', 'METFORMIN'],
 ['SITAGLIPTIN', 'METFORMIN'],
 ['METFORMIN', 'HYDROCHLORIDE'],
 ['VILDAGLIPTIN', 'METFORMIN'],
 ['METFORMIN'],
 ['SITAGLIPTIN', 'METFORMIN'],
 ['LINAGLIPTIN', 'METFORMIN'],
 ['DAPAGLIFLOZIN', 'METFORMIN'],
 ['VILDAGLIPTIN', 'METFORM

### part 3 - approach 2 (using simhash calculate distance)

In [32]:
# try to pair the againnnnnn

#!pip install simhash

from simhash import Simhash

In [72]:
pairIndexlist = []
for dirtyname in CHLDiab1DirtyNameIndexEng:
  distance_list = []
  for index in MainMedicineIndex:
    distance = Simhash(index).distance(Simhash(dirtyname))
    distance_list.append(distance)
  index_min = min(range(len(distance_list)), key=distance_list.__getitem__) # get the index which correspond to the minimum distance
  add_index = MainMedicineIndex[index_min]
  pairIndexlist.append(add_index)

In [81]:
PairIndexData = {"Chile 1 drug names": CHLDiab1DirtyNameIndexEng, "Catalog Index": pairIndexlist}

In [82]:
print(len(CHLDiab1DirtyNameIndexEng), len(pairIndexlist))

3663 3663


In [83]:
PairIndexData_df = pd.DataFrame(PairIndexData)

In [84]:
PairIndexData_df

,Chile 1 drug names,Catalog Index
0,"[METFORMIN, HYDROCHLORIDE, TABLET, COVERED, EQ...","[METFORMIN, HYDROCHLORIDE]"
1,"[METFORMIN, TABLETS, RELEASE, EXTENDED, CONTAI...","[SITAGLIPTIN, METFORMIN]"
2,"[METFORMIN, TABLETS, COATED]",[METFORMIN]
3,"[INSUÑLINA, DEGLUDEC, SIMILAR, TRESIBA, TOUCH]",[AZATHIOPRINE]
4,"[DAPAGLIFLOZINA, PROPANODIOL, METFORMIN, HYDRO...","[METFORMIN, HYDROCHLORIDE]"
...,...,...
3658,"[METFORMIN, 1000MG, PROLONG, BIOEQUIVALENTE]","[SITAGLIPTIN, METFORMIN]"
3659,"[INSULIN, GLULISINA, PENCIL, APIDRA, SIMILAR, ...","[LAMIVUDINE, ZIDOVUDINE]"
3660,"[METFORMIN, TABLET]","[GLIBENCLAMIDE, METFORMIN]"
3661,"[BOXES, TABLETS, METFORMIN, VILDAGLIPTINA, SUG...","[SITAGLIPTIN, METFORMIN]"


## Part 4: combine the Chile drug name index pair data frame and full catagol name index data frame to get the result

In [101]:
# for Chile pair index dataframe, transform list to string
str_pairIndexlist = []
for item in pairIndexlist:
  newitem = ' '.join(item)
  str_pairIndexlist.append(newitem)

In [102]:
str_CHLDiab1DirtyNameIndexEng = []
for item in CHLDiab1DirtyNameIndexEng:
  newitem = ' '.join(item)
  str_CHLDiab1DirtyNameIndexEng.append(newitem)

In [103]:
newPairIndexData = {"Chile 1 drug names": str_CHLDiab1DirtyNameIndexEng, "Catalog Index": str_pairIndexlist}

In [104]:
newPairIndexData_df = pd.DataFrame(newPairIndexData)

In [105]:
newPairIndexData_df[:2]

,Chile 1 drug names,Catalog Index
0,METFORMIN HYDROCHLORIDE TABLET COVERED EQUIVAL...,METFORMIN HYDROCHLORIDE
1,METFORMIN TABLETS RELEASE EXTENDED CONTAINER T...,SITAGLIPTIN METFORMIN


In [106]:
# for catalog name index dataframe, transform list to string
str_MainMedicineIndex = []
for item in MainMedicineIndex:
  newitem = ' '.join(item)
  str_MainMedicineIndex.append(newitem)

In [107]:
newCatalogIndexData = {"Catalog Name": CatalogNameU, "Catalog Index": str_MainMedicineIndex}

In [108]:
newCatalogIndexData_df = pd.DataFrame(newCatalogIndexData)

In [109]:
newCatalogIndexData_df[:2]

,Catalog Name,Catalog Index
0,"AMLODIPINO 5 MG + VALSARTAN 80 MG, TABLET, TBD",AMLODIPINO VALSARTAN
1,"GLIBENCLAMIDE 5 MG + METFORMIN 500 MG, TABLET,...",GLIBENCLAMIDE METFORMIN


In [112]:
combine_pair_df = pd.merge(newPairIndexData_df, newCatalogIndexData_df, on='Catalog Index', how='left')

In [113]:
combine_pair_df.to_excel("combine_pair_df_chile_1.xlsx", sheet_name = 'combine_pair_df chile 1')  